## pre_process.ipynb

- 数据预处理

## 生成数据描述

1. ccf_off_test.csv

- 由 dataset_raw/ccf_offline_stage1_test_revised.csv 生成

- 新增列：

| no_distance      | is_full_discount | discount_x      | discount_y      | discount_rate    | discount_type          |
| ---------------- | ---------------- | --------------- | --------------- | ---------------- | ---------------------- |
| 是否没有距离信息 | 是否是满减优惠券 | 满减时满多少RMB | 满减时减多少RMB | 优惠券等价折扣率 | 优惠券种类硬编码(17种) |

2. ccf_off_train_csv

- 由 dataset_raw/ccf_offline_stage1_train.csv 生成

- 在 `1` 的基础上新增列:

| normal_consume                     | coupon_consume                     | no_consume                 |
| ---------------------------------- | ---------------------------------- | -------------------------- |
| 是否是没有使用优惠券消费(正常消费) | 是否是使用了优惠券消费(无15天限制) | 是否是领了优惠券但没有消费 |

---

> 三者类似独热编码，只有也一定会有一个是 1，剩下两个是 0
> (没有领优惠券并且没有消费不需要记录)

3. ccf_on_train.csv

- 由 dataset_raw/ccf_online_stage1_train.csv 生成

- 与 `2` 相比，新增特征有如下不同

  - fixed_consume: 是否是限时降价的消费 (限时降价的数据都消费了)
  - normal_consume，coupon_consume，no_consume，fixed_consume 四者之和不一定为 1
    - 不为 1 的 Action 一定为 0     (点击行为)
    - Action 为 0 的四者之和不为 1
    - 点击行为没有优惠券
    - 没有优惠券的不一定是点击行为，还有一个正常消费行为
  - discount_rate 为 -1.0 时表示限时降价消费
    - 点击和正常购买时 discount_rate 都等于 1.0

In [1]:
import pandas as pd
import numpy as np

no_date = pd.to_datetime(0)  # 时间戳零点

In [4]:
test = pd.read_csv('./dataset_raw/ccf_offline_stage1_test_revised.csv', parse_dates=["Date_received"])
test

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1.0,2016-07-12
1,6949378,1300,3429,30:5,NaN,2016-07-06
2,2166529,7113,6928,200:20,5.0,2016-07-27
3,2166529,7113,1808,100:10,5.0,2016-07-27
4,6172162,7605,6500,30:1,2.0,2016-07-08
...,...,...,...,...,...,...
113635,5828093,5717,10418,30:5,10.0,2016-07-16
113636,6626813,1699,7595,30:1,NaN,2016-07-07
113637,6626813,7321,7590,50:5,NaN,2016-07-12
113638,4547069,760,13602,30:5,0.0,2016-07-17


In [5]:
test.describe()

,User_id,Merchant_id,Coupon_id,Distance
count,1.136400e+05,113640.000000,113640.000000,101576.000000
mean,3.684858e+06,2962.283853,9053.810929,2.328040
std,2.126259e+06,2494.450802,4145.873088,3.260755
min,2.090000e+02,6.000000,3.000000,0.000000
25%,1.844191e+06,760.000000,5023.000000,0.000000
50%,3.683266e+06,2050.000000,9983.000000,1.000000
75%,5.525845e+06,5138.000000,13602.000000,3.000000
max,7.361024e+06,8856.000000,14045.000000,10.000000


In [6]:
test.notna().all()

User_id           True
Merchant_id       True
Coupon_id         True
Discount_rate     True
Distance         False
Date_received     True
dtype: bool

In [7]:
test['Distance'] = test['Distance'].fillna(-1).astype(int)

In [8]:
test

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1,2016-07-12
1,6949378,1300,3429,30:5,-1,2016-07-06
2,2166529,7113,6928,200:20,5,2016-07-27
3,2166529,7113,1808,100:10,5,2016-07-27
4,6172162,7605,6500,30:1,2,2016-07-08
...,...,...,...,...,...,...
113635,5828093,5717,10418,30:5,10,2016-07-16
113636,6626813,1699,7595,30:1,-1,2016-07-07
113637,6626813,7321,7590,50:5,-1,2016-07-12
113638,4547069,760,13602,30:5,0,2016-07-17


In [9]:
test['no_distance'] = (test['Distance'] == -1).astype(int)
test['is_full_discount'] = test['Discount_rate'].str.contains(':').astype(int)
test[['discount_x', 'discount_y']] = test[test['is_full_discount'] == 1]['Discount_rate']\
    .str.split(':', expand=True).astype(float)
     # expand 设置成 true 才可以返回一个 dataframe，设置成 float 是因为合并时有NA
test['discount_rate'] = (1 - (test['discount_y'] / test['discount_x']))\
    .fillna(test['Discount_rate']).astype(float)
test[['discount_x', 'discount_y']] = \
    test[['discount_x', 'discount_y']].fillna(-1).astype(int)

In [10]:
rate = sorted(set(test.discount_rate))
test['discount_type'] = test['discount_rate'].apply(lambda x: rate.index(x))

In [11]:
test

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,no_distance,is_full_discount,discount_x,discount_y,discount_rate,discount_type
0,4129537,450,9983,30:5,1,2016-07-12,0,1,30,5,0.833333,7
1,6949378,1300,3429,30:5,-1,2016-07-06,1,1,30,5,0.833333,7
2,2166529,7113,6928,200:20,5,2016-07-27,0,1,200,20,0.900000,10
3,2166529,7113,1808,100:10,5,2016-07-27,0,1,100,10,0.900000,10
4,6172162,7605,6500,30:1,2,2016-07-08,0,1,30,1,0.966667,14
...,...,...,...,...,...,...,...,...,...,...,...,...
113635,5828093,5717,10418,30:5,10,2016-07-16,0,1,30,5,0.833333,7
113636,6626813,1699,7595,30:1,-1,2016-07-07,1,1,30,1,0.966667,14
113637,6626813,7321,7590,50:5,-1,2016-07-12,1,1,50,5,0.900000,10
113638,4547069,760,13602,30:5,0,2016-07-17,0,1,30,5,0.833333,7


In [12]:
test.notna().all()

User_id             True
Merchant_id         True
Coupon_id           True
Discount_rate       True
Distance            True
Date_received       True
no_distance         True
is_full_discount    True
discount_x          True
discount_y          True
discount_rate       True
discount_type       True
dtype: bool

In [13]:
test.describe()

,User_id,Merchant_id,Coupon_id,Distance,no_distance,is_full_discount,discount_x,discount_y,discount_rate,discount_type
count,1.136400e+05,113640.000000,113640.000000,113640.000000,113640.000000,113640.000000,113640.000000,113640.000000,113640.000000,113640.000000
mean,3.684858e+06,2962.283853,9053.810929,1.974736,0.106160,0.977420,43.396507,6.089141,0.850671,8.146920
std,2.126259e+06,2494.450802,4145.873088,3.248809,0.308043,0.148561,44.239815,5.640310,0.063551,2.631381
min,2.090000e+02,6.000000,3.000000,-1.000000,0.000000,0.000000,-1.000000,-1.000000,0.333333,0.000000
25%,1.844191e+06,760.000000,5023.000000,0.000000,0.000000,1.000000,30.000000,5.000000,0.833333,7.000000
50%,3.683266e+06,2050.000000,9983.000000,1.000000,0.000000,1.000000,30.000000,5.000000,0.833333,7.000000
75%,5.525845e+06,5138.000000,13602.000000,3.000000,0.000000,1.000000,30.000000,5.000000,0.900000,10.000000
max,7.361024e+06,8856.000000,14045.000000,10.000000,1.000000,1.000000,500.000000,100.000000,0.990000,17.000000


In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113640 entries, 0 to 113639
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   User_id           113640 non-null  int64         
 1   Merchant_id       113640 non-null  int64         
 2   Coupon_id         113640 non-null  int64         
 3   Discount_rate     113640 non-null  object        
 4   Distance          113640 non-null  int64         
 5   Date_received     113640 non-null  datetime64[ns]
 6   no_distance       113640 non-null  int64         
 7   is_full_discount  113640 non-null  int64         
 8   discount_x        113640 non-null  int64         
 9   discount_y        113640 non-null  int64         
 10  discount_rate     113640 non-null  float64       
 11  discount_type     113640 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(9), object(1)
memory usage: 10.4+ MB


In [15]:
test.to_csv('./dataset_cleaned/ccf_off_test.csv', index=None)

+ 生成一个函数

In [16]:
def pre_process_off_test(df: pd.DataFrame):
    """
    测试线下数据集处理，没有 Date
    """
    test['Distance'] = test['Distance'].fillna(-1).astype(int)
    test['no_distance'] = (test['Distance'] == -1).astype(int)
    test['is_full_discount'] = test['Discount_rate'].str.contains(':').astype(int)
    test[['discount_x', 'discount_y']] = test[test['is_full_discount'] == 1]['Discount_rate']\
        .str.split(':', expand=True).astype(float)
     # expand 设置成 true 才可以返回一个 dataframe，设置成 float 是因为合并时有NA
    test['discount_rate'] = (1 - (test['discount_y'] / test['discount_x']))\
        .fillna(test['Discount_rate']).astype(float)
    test[['discount_x', 'discount_y']] = \
        test[['discount_x', 'discount_y']].fillna(-1).astype(int)
    _rate = sorted(set(test.discount_rate))
    test['discount_type'] = test['discount_rate'].apply(lambda x: _rate.index(x))
    return df

In [17]:
train_off = pd.read_csv('./dataset_raw/ccf_offline_stage1_train.csv', parse_dates=['Date', 'Date_received'])

In [18]:
train_off.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754884 entries, 0 to 1754883
Data columns (total 7 columns):
 #   Column         Dtype         
---  ------         -----         
 0   User_id        int64         
 1   Merchant_id    int64         
 2   Coupon_id      float64       
 3   Discount_rate  object        
 4   Distance       float64       
 5   Date_received  datetime64[ns]
 6   Date           datetime64[ns]
dtypes: datetime64[ns](2), float64(2), int64(2), object(1)
memory usage: 93.7+ MB


In [19]:
train_off.notna().all()
# 赛题
# Coupon_id null表示无优惠券消费，此时Discount_rate和Date_received字段无意义
# Date 如果Date=null & Coupon_id != null，该记录表示领取优惠券但没有使用，即负样本；
# 如果Date!=null & Coupon_id = null，则表示普通消费日期；如果Date!=null & Coupon_id != null，
# 则表示用优惠券消费日期，即正样本； 

User_id           True
Merchant_id       True
Coupon_id        False
Discount_rate    False
Distance         False
Date_received    False
Date             False
dtype: bool

In [20]:
train_off[train_off['Date_received'].notna()]['Coupon_id'].notna().all(), \
    train_off[train_off['Coupon_id'].notna()]['Date_received'].notna().all()
# Date_received 和 Coupon_id 同时 NA

(True, True)

In [21]:
train_off[train_off['Discount_rate'].notna()]['Coupon_id'].notna().all(), \
    train_off[train_off['Coupon_id'].notna()]['Discount_rate'].notna().all()
# Discount_rate 和 Coupon_id 同时 NA

# Date_received, Discount_rate, Coupon_id 同时 NA

(True, True)

In [22]:
train_off['normal_consume'] = 0  # 加上是否是正常消费
train_off.loc[train_off['Coupon_id'].isna() & train_off['Date'].notna(), 'normal_consume'] = 1

In [23]:
len(train_off[train_off['normal_consume'] == 1]) / len(train_off) # 正常消费占 0.4

0.39979964487681235

In [24]:
train_off['coupon_consume'] = 0  # 是否是使用优惠券消费 (没有15天限制)
train_off.loc[train_off['Coupon_id'].notna() & train_off['Date'].notna(), 'coupon_consume'] = 1

In [25]:
len(train_off[train_off['coupon_consume'] == 1]) / len(train_off) # 使用优惠券消费的占 0.04

0.04295554577966407

In [26]:
train_off['no_consume'] = 0  # 领了优惠券但没有消费
train_off.loc[train_off['Coupon_id'].notna() & train_off['Date'].isna(), 'no_consume'] = 1

In [27]:
len(train_off[train_off['no_consume'] == 1]) / len(train_off) # 白嫖怪不消费的占 0.55

0.5572448093435236

In [28]:
len(train_off[train_off['normal_consume'] == 1]) / len(train_off) +\
    len(train_off[train_off['coupon_consume'] == 1]) / len(train_off) +\
        len(train_off[train_off['no_consume'] == 1]) / len(train_off)
# 等于 1 说明没有数据是 `未领优惠券也未消费的`，很显然每一个`局外人`都是这一类，不需要统计

1.0

In [29]:
train_off

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,normal_consume,coupon_consume,no_consume
0,1439408,2632,NaN,NaN,0.0,NaT,2016-02-17,1,0,0
1,1439408,4663,11002.0,150:20,1.0,2016-05-28,NaT,0,0,1
2,1439408,2632,8591.0,20:1,0.0,2016-02-17,NaT,0,0,1
3,1439408,2632,1078.0,20:1,0.0,2016-03-19,NaT,0,0,1
4,1439408,2632,8591.0,20:1,0.0,2016-06-13,NaT,0,0,1
...,...,...,...,...,...,...,...,...,...,...
1754879,212662,3532,NaN,NaN,1.0,NaT,2016-03-22,1,0,0
1754880,212662,3021,3739.0,30:1,6.0,2016-05-08,2016-06-02,0,1,0
1754881,212662,2934,NaN,NaN,2.0,NaT,2016-03-21,1,0,0
1754882,752472,7113,1633.0,50:10,6.0,2016-06-13,NaT,0,0,1


In [30]:
train_off['Coupon_id'].fillna(0, inplace=True)
train_off['Discount_rate'].fillna(1.0, inplace=True)  # 没有优惠券消费相当于10折
train_off['Distance'].fillna(-1, inplace=True)
train_off['Date_received'].fillna(no_date, inplace=True)
train_off['Date'].fillna(no_date, inplace=True)

In [31]:
train_off

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,normal_consume,coupon_consume,no_consume
0,1439408,2632,0.0,1.0,0.0,1970-01-01,2016-02-17,1,0,0
1,1439408,4663,11002.0,150:20,1.0,2016-05-28,1970-01-01,0,0,1
2,1439408,2632,8591.0,20:1,0.0,2016-02-17,1970-01-01,0,0,1
3,1439408,2632,1078.0,20:1,0.0,2016-03-19,1970-01-01,0,0,1
4,1439408,2632,8591.0,20:1,0.0,2016-06-13,1970-01-01,0,0,1
...,...,...,...,...,...,...,...,...,...,...
1754879,212662,3532,0.0,1.0,1.0,1970-01-01,2016-03-22,1,0,0
1754880,212662,3021,3739.0,30:1,6.0,2016-05-08,2016-06-02,0,1,0
1754881,212662,2934,0.0,1.0,2.0,1970-01-01,2016-03-21,1,0,0
1754882,752472,7113,1633.0,50:10,6.0,2016-06-13,1970-01-01,0,0,1


In [32]:
train_off.notna().all()

User_id           True
Merchant_id       True
Coupon_id         True
Discount_rate     True
Distance          True
Date_received     True
Date              True
normal_consume    True
coupon_consume    True
no_consume        True
dtype: bool

+ 结合上面的函数封装一个新的函数

In [2]:
def pre_process_off_new(df: pd.DataFrame):
    """线下训练数据集，有 Date"""
    if 'Date' in df.columns:  
        df['normal_consume'] = 0  # 加上是否是正常消费
        df.loc[df['Coupon_id'].isna() & df['Date'].notna(), 'normal_consume'] = 1
        df['coupon_consume'] = 0  # 是否是使用优惠券消费 (没有15天限制)
        df.loc[df['Coupon_id'].notna() & df['Date'].notna(), 'coupon_consume'] = 1
        df['no_consume'] = 0  # 领了优惠券但没有消费
        df.loc[df['Coupon_id'].notna() & df['Date'].isna(), 'no_consume'] = 1
        df['Coupon_id'] = df['Coupon_id'].fillna(0).astype(int) # Coupon_id 由 nullable 转换成 notnull 会把整数类型转成 float，这里转回去
        df['Discount_rate'].fillna('1.0', inplace=True)  # 没有优惠券消费相当于10折，这里得填 str 下面类型才不会出问题
        # df['Distance'].fillna(-1, inplace=True)  Distance 下面就可以处理
        df['Date_received'].fillna(no_date, inplace=True)
        df['Date'].fillna(no_date, inplace=True)
    
    '''
    线下测试数据集处理，没有 Date
    '''
    df['Distance'] = df['Distance'].fillna(-1).astype(int)
    df['no_distance'] = (df['Distance'] == -1).astype(int)
    df['is_full_discount'] = df['Discount_rate'].str.contains(':').astype(int)
    df[['discount_x', 'discount_y']] = df[df['is_full_discount'] == 1]['Discount_rate']\
        .str.split(':', expand=True).astype(float)
     # expand 设置成 true 才可以返回一个 dataframe，设置成 float 是因为合并时有NA
    df['discount_rate'] = (1 - (df['discount_y'] / df['discount_x']))\
        .fillna(df['Discount_rate']).astype(float)
    df[['discount_x', 'discount_y']] = \
        df[['discount_x', 'discount_y']].fillna(-1).astype(int)
    _rate = sorted(set(df.discount_rate))
    df['discount_type'] = df['discount_rate'].apply(lambda x: _rate.index(x))
    return df

In [3]:
test_df = pd.read_csv('./dataset_raw/ccf_offline_stage1_test_revised.csv', parse_dates=['Date_received'])
train_off_df = pd.read_csv('./dataset_raw/ccf_offline_stage1_train.csv', parse_dates=['Date_received', 'Date'])

In [4]:
out_test_df = pre_process_off_new(test_df)
out_train_off_df = pre_process_off_new(train_off_df)

In [5]:
out_test_df.notna().all().all(), out_train_off_df.notna().all().all()

(True, True)

In [6]:
out_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113640 entries, 0 to 113639
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   User_id           113640 non-null  int64         
 1   Merchant_id       113640 non-null  int64         
 2   Coupon_id         113640 non-null  int64         
 3   Discount_rate     113640 non-null  object        
 4   Distance          113640 non-null  int64         
 5   Date_received     113640 non-null  datetime64[ns]
 6   no_distance       113640 non-null  int64         
 7   is_full_discount  113640 non-null  int64         
 8   discount_x        113640 non-null  int64         
 9   discount_y        113640 non-null  int64         
 10  discount_rate     113640 non-null  float64       
 11  discount_type     113640 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(9), object(1)
memory usage: 10.4+ MB


In [7]:
out_train_off_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754884 entries, 0 to 1754883
Data columns (total 16 columns):
 #   Column            Dtype         
---  ------            -----         
 0   User_id           int64         
 1   Merchant_id       int64         
 2   Coupon_id         int64         
 3   Discount_rate     object        
 4   Distance          int64         
 5   Date_received     datetime64[ns]
 6   Date              datetime64[ns]
 7   normal_consume    int64         
 8   coupon_consume    int64         
 9   no_consume        int64         
 10  no_distance       int64         
 11  is_full_discount  int64         
 12  discount_x        int64         
 13  discount_y        int64         
 14  discount_rate     float64       
 15  discount_type     int64         
dtypes: datetime64[ns](2), float64(1), int64(12), object(1)
memory usage: 214.2+ MB


In [8]:
(out_train_off_df['normal_consume']
 + out_train_off_df['coupon_consume']
 + out_train_off_df['no_consume'] == 1).all()
# 类似独热编码

True

In [9]:
out_test_df.to_csv('./dataset_cleaned/ccf_off_test.csv', index=None)
out_train_off_df.to_csv('./dataset_cleaned/ccf_off_train.csv', index=None)

- 线上特征处理

In [234]:
on = pd.read_csv('./dataset_raw/ccf_online_stage1_train.csv', parse_dates=['Date', 'Date_received'])

In [235]:
on

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date
0,13740231,18907,2,100017492,500:50,2016-05-13,NaT
1,13740231,34805,1,NaN,NaN,NaT,2016-03-21
2,14336199,18907,0,NaN,NaN,NaT,2016-06-18
3,14336199,18907,0,NaN,NaN,NaT,2016-06-18
4,14336199,18907,0,NaN,NaN,NaT,2016-06-18
...,...,...,...,...,...,...,...
11429821,13087731,27715,0,NaN,NaN,NaT,2016-06-29
11429822,13087731,52005,0,NaN,NaN,NaT,2016-03-24
11429823,13087731,45611,0,NaN,NaN,NaT,2016-04-22
11429824,13683699,18009,1,NaN,NaN,NaT,2016-03-23


In [237]:
on.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11429826 entries, 0 to 11429825
Data columns (total 7 columns):
 #   Column         Dtype         
---  ------         -----         
 0   User_id        int64         
 1   Merchant_id    int64         
 2   Action         int64         
 3   Coupon_id      object        
 4   Discount_rate  object        
 5   Date_received  datetime64[ns]
 6   Date           datetime64[ns]
dtypes: datetime64[ns](2), int64(3), object(2)
memory usage: 610.4+ MB


In [238]:
on.notna().all()
# Coupon_id: null表示无优惠券消费，此时Discount_rate和Date_received字段无意义。
# “fixed”表示该交易是限时低价活动。 
# Discount_rate: x \in [0,1]代表折扣率；x:y表示满x减y；“fixed”表示低价限时优惠； 
# Action: 0 点击， 1购买，2领取优惠券 
# Date: 如果Date=null & Coupon_id != null，该记录表示领取优惠券但没有使用(此时 Action=2)；
# 如果Date!=null & Coupon_id = null，则表示普通消费/点击日期(Action=0或1)；
# 如果Date!=null & Coupon_id != null，则表示用优惠券消费/fixed消费日期(Action=1)；

User_id           True
Merchant_id       True
Action            True
Coupon_id        False
Discount_rate    False
Date_received    False
Date             False
dtype: bool

In [239]:
on[on['Action'] == 0]

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date
2,14336199,18907,0,NaN,NaN,NaT,2016-06-18
3,14336199,18907,0,NaN,NaN,NaT,2016-06-18
4,14336199,18907,0,NaN,NaN,NaT,2016-06-18
5,14336199,18907,0,NaN,NaN,NaT,2016-06-18
6,14336199,18907,0,NaN,NaN,NaT,2016-06-18
...,...,...,...,...,...,...,...
11429819,12985299,10813,0,NaN,NaN,NaT,2016-04-14
11429820,13087731,52509,0,NaN,NaN,NaT,2016-06-09
11429821,13087731,27715,0,NaN,NaN,NaT,2016-06-29
11429822,13087731,52005,0,NaN,NaN,NaT,2016-03-24


In [242]:
on[on['Action'] == 0]['Coupon_id'].isna().all(), \
    on[on['Action'] == 0]['Discount_rate'].isna().all(), \
        on[on['Action'] == 0]['Date_received'].isna().all()
# 点击时上面三者均为 na

(True, True, True)

In [243]:
on[on['Action'] == 0]['Date'].notna().all()
# 点击时 Date 全部都不是 na
# 表示点击

True

In [247]:
on[on['Action'] == 2]

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date
0,13740231,18907,2,100017492,500:50,2016-05-13,NaT
73,15137031,25104,2,100120354,150:50,2016-06-13,NaT
74,15137031,44706,2,100071973,50:5,2016-03-17,NaT
114,15137031,29007,2,100028000,30:1,2016-04-05,NaT
115,15137031,18907,2,100086665,300:50,2016-04-06,NaT
...,...,...,...,...,...,...,...
11429600,13495131,45902,2,100014418,150:5,2016-04-20,NaT
11429601,13495131,45902,2,100176168,200:10,2016-04-20,NaT
11429602,14091099,44805,2,100197854,5:1,2016-04-27,NaT
11429625,14091099,47412,2,100148588,30:20,2016-05-31,NaT


In [245]:
on[on['Action'] == 2]['Date'].isna().all()
# 领取优惠券时全没有消费日期

True

In [252]:
on[on['Action'] == 1]

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date
1,13740231,34805,1,NaN,NaN,NaT,2016-03-21
19,10539231,12008,1,NaN,NaN,NaT,2016-06-18
21,10539231,12008,1,NaN,NaN,NaT,2016-06-18
75,15137031,18206,1,NaN,NaN,NaT,2016-05-14
76,15137031,18206,1,NaN,NaN,NaT,2016-05-14
...,...,...,...,...,...,...,...
11429791,12985299,49800,1,100181877,10:5,2016-05-16,2016-05-17
11429810,12985299,49800,1,fixed,fixed,2016-05-17,2016-05-17
11429818,12985299,49800,1,NaN,NaN,NaT,2016-05-20
11429824,13683699,18009,1,NaN,NaN,NaT,2016-03-23


In [249]:
on[on['Action'] == 1]['Date'].notna().all()

True

In [260]:
on['coupon_consume'] = 0  # 是否使用了优惠券消费
on.loc[on['Date'].notna() & on['Coupon_id'].notna() & (on['Coupon_id'] != 'fixed'),\
     'coupon_consume'] = 1

In [272]:
on['fixed_consume'] = 0  # 是否是限时降价消费
on.loc[on['Date'].notna() & (on['Coupon_id'] == 'fixed'), 'fixed_consume'] = 1

In [278]:
(on[on['Date'].notna() & (on['Coupon_id'] == 'fixed')]['Date_received']
 == on[on['Date'].notna() & (on['Coupon_id'] == 'fixed')]['Date']).all()
# 限时降价的 Date_received 就是 Date，没有意义

True

In [279]:
on.loc[on['Date'].notna() & (on['Coupon_id'] == 'fixed'), 'Date_received'] = no_date
# 移除 fixed 的 Date_received

In [280]:
on[on['Date'].notna() & (on['Coupon_id'] == 'fixed')]

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date,coupon_consume,fixed_consume
273,10131831,47003,1,fixed,fixed,1970-01-01,2016-01-28,0,1
274,10131831,47003,1,fixed,fixed,1970-01-01,2016-01-28,0,1
279,10131831,47003,1,fixed,fixed,1970-01-01,2016-01-17,0,1
280,10131831,47003,1,fixed,fixed,1970-01-01,2016-01-17,0,1
748,12518031,30314,1,fixed,fixed,1970-01-01,2016-03-06,0,1
...,...,...,...,...,...,...,...,...,...
11428992,12214731,51810,1,fixed,fixed,1970-01-01,2016-03-07,0,1
11429163,15008331,46602,1,fixed,fixed,1970-01-01,2016-02-18,0,1
11429496,15299331,18907,1,fixed,fixed,1970-01-01,2016-03-08,0,1
11429499,15299331,18907,1,fixed,fixed,1970-01-01,2016-03-19,0,1


In [288]:
on['normal_consume'] = 0  # 是否是正常消费，没有使用优惠券的消费行为
on.loc[(on['Action'] == 1) & on['Coupon_id'].isna(), 'normal_consume'] = 1

In [287]:
len(on[on['Date'].notna() & on['Coupon_id'].isna()]) == \
    len(on[(on['Action'] == 0) & on['Coupon_id'].isna()]) \
        + len(on[(on['Action'] == 1) & on['Coupon_id'].isna()])
# Date 不是 na 是，Coupon_id 是 na 只有两种情况：点击/购买

True

In [294]:
on['no_consume'] = 0  # 领取了优惠券但没有消费
on.loc[(on['Action'] == 2), 'no_consume'] = 1

In [298]:
tmp = on[(on['coupon_consume'] + on['fixed_consume'] + on['normal_consume'] + on['no_consume'] != 1)]
# 四者之和不为1的

In [303]:
len(tmp) == len(on[on['Action'] == 0])
# 全是点击行为

True

In [305]:
on.notna().all()

User_id            True
Merchant_id        True
Action             True
Coupon_id         False
Discount_rate     False
Date_received     False
Date              False
coupon_consume     True
fixed_consume      True
normal_consume     True
no_consume         True
dtype: bool

In [309]:
(on[on['Date'].isna()]['Action'] == 2).all()
# Date 是 na 的全是领取优惠券但没有使用的数据

True

In [310]:
on

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date,coupon_consume,fixed_consume,normal_consume,no_consume
0,13740231,18907,2,100017492,500:50,2016-05-13,NaT,0,0,0,1
1,13740231,34805,1,NaN,NaN,NaT,2016-03-21,0,0,1,0
2,14336199,18907,0,NaN,NaN,NaT,2016-06-18,0,0,0,0
3,14336199,18907,0,NaN,NaN,NaT,2016-06-18,0,0,0,0
4,14336199,18907,0,NaN,NaN,NaT,2016-06-18,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
11429821,13087731,27715,0,NaN,NaN,NaT,2016-06-29,0,0,0,0
11429822,13087731,52005,0,NaN,NaN,NaT,2016-03-24,0,0,0,0
11429823,13087731,45611,0,NaN,NaN,NaT,2016-04-22,0,0,0,0
11429824,13683699,18009,1,NaN,NaN,NaT,2016-03-23,0,0,1,0


In [313]:
on['Date'].fillna(no_date, inplace=True)
on['Date_received'].fillna(no_date, inplace=True)
on['Coupon_id'] = on['Coupon_id'].replace('fixed', 0)
on['Coupon_id'] = on['Coupon_id'].fillna(0).astype(int)
on['Discount_rate'] = on['Discount_rate'].replace('fixed', np.random.uniform(low=0.8, high=1.0)) # Discount_rate 随机初始化
on['Discount_rate'].fillna('1.0', inplace=True)

In [315]:
on.notna().all()

User_id           True
Merchant_id       True
Action            True
Coupon_id         True
Discount_rate     True
Date_received     True
Date              True
coupon_consume    True
fixed_consume     True
normal_consume    True
no_consume        True
dtype: bool

In [316]:
on.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11429826 entries, 0 to 11429825
Data columns (total 11 columns):
 #   Column          Dtype         
---  ------          -----         
 0   User_id         int64         
 1   Merchant_id     int64         
 2   Action          int64         
 3   Coupon_id       int64         
 4   Discount_rate   object        
 5   Date_received   datetime64[ns]
 6   Date            datetime64[ns]
 7   coupon_consume  int64         
 8   fixed_consume   int64         
 9   normal_consume  int64         
 10  no_consume      int64         
dtypes: datetime64[ns](2), int64(8), object(1)
memory usage: 959.2+ MB


- 封装一个函数

In [10]:
def pre_process_online(df: pd.DataFrame):
    df['coupon_consume'] = 0  # 是否使用了优惠券消费
    df.loc[df['Date'].notna() & df['Coupon_id'].notna() & (df['Coupon_id'] != 'fixed'),\
        'coupon_consume'] = 1
    df['fixed_consume'] = 0  # 是否是限时降价消费
    df.loc[df['Date'].notna() & (df['Coupon_id'] == 'fixed'), 'fixed_consume'] = 1
    # 移除 fixed 的 Date_received
    df.loc[df['Date'].notna() & (df['Coupon_id'] == 'fixed'), 'Date_received'] = no_date
    df['normal_consume'] = 0  # 是否是正常消费，没有使用优惠券的消费行为
    df.loc[(df['Action'] == 1) & df['Coupon_id'].isna(), 'normal_consume'] = 1
    df['no_consume'] = 0  # 是否是领取了优惠券但没有消费
    df.loc[(df['Action'] == 2), 'no_consume'] = 1
    df['Date'].fillna(no_date, inplace=True)
    df['Date_received'].fillna(no_date, inplace=True)
    df['Coupon_id'] = df['Coupon_id'].replace('fixed', 0)
    df['Coupon_id'] = df['Coupon_id'].fillna(0).astype(int)
    df['Discount_rate'] = df['Discount_rate'].replace('fixed', '-1.0') # 标记为 -1.0
    df['Discount_rate'].fillna('1.0', inplace=True)
    df['is_full_discount'] = df['Discount_rate'].str.contains(':').astype(int)
    df[['discount_x', 'discount_y']] = df[df['is_full_discount'] == 1]['Discount_rate']\
        .str.split(':', expand=True).astype(float)
     # expand 设置成 true 才可以返回一个 dataframe，设置成 float 是因为合并时有NA
    df['discount_rate'] = (1 - (df['discount_y'] / df['discount_x']))\
        .fillna(df['Discount_rate']).astype(float)
    df[['discount_x', 'discount_y']] = \
        df[['discount_x', 'discount_y']].fillna(-1).astype(int)
    _rate = sorted(set(df.discount_rate))
    df['discount_type'] = df['discount_rate'].apply(lambda x: _rate.index(x))
    return df

In [11]:
on_data = pd.read_csv('./dataset_raw/ccf_online_stage1_train.csv', parse_dates=['Date', 'Date_received'])

In [12]:
out_on_data = pre_process_online(on_data)

In [13]:
out_on_data

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date,coupon_consume,fixed_consume,normal_consume,no_consume,is_full_discount,discount_x,discount_y,discount_rate,discount_type
0,13740231,18907,2,100017492,500:50,2016-05-13,1970-01-01,0,0,0,1,1,500,50,0.9,14
1,13740231,34805,1,0,1.0,1970-01-01,2016-03-21,0,0,1,0,0,-1,-1,1.0,31
2,14336199,18907,0,0,1.0,1970-01-01,2016-06-18,0,0,0,0,0,-1,-1,1.0,31
3,14336199,18907,0,0,1.0,1970-01-01,2016-06-18,0,0,0,0,0,-1,-1,1.0,31
4,14336199,18907,0,0,1.0,1970-01-01,2016-06-18,0,0,0,0,0,-1,-1,1.0,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11429821,13087731,27715,0,0,1.0,1970-01-01,2016-06-29,0,0,0,0,0,-1,-1,1.0,31
11429822,13087731,52005,0,0,1.0,1970-01-01,2016-03-24,0,0,0,0,0,-1,-1,1.0,31
11429823,13087731,45611,0,0,1.0,1970-01-01,2016-04-22,0,0,0,0,0,-1,-1,1.0,31
11429824,13683699,18009,1,0,1.0,1970-01-01,2016-03-23,0,0,1,0,0,-1,-1,1.0,31


In [14]:
out_on_data.notna().all()

User_id             True
Merchant_id         True
Action              True
Coupon_id           True
Discount_rate       True
Date_received       True
Date                True
coupon_consume      True
fixed_consume       True
normal_consume      True
no_consume          True
is_full_discount    True
discount_x          True
discount_y          True
discount_rate       True
discount_type       True
dtype: bool

In [15]:
out_on_data.to_csv('./dataset_cleaned/ccf_on_train.csv', index=None)